<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/(%D0%91%D1%8B%D1%81%D1%82%D1%80%D1%8B%D0%B9)_%D0%94%D0%B5%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80_%D0%BB%D0%B8%D1%86%D0%B5%D0%B2%D1%8B%D1%85_%D0%BE%D1%80%D0%B8%D0%B5%D0%BD%D1%82%D0%B8%D1%80%D0%BE%D0%B2_%D1%81_dlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (Быстрый) Детектор лицевых ориентиров с dlib

# (Faster) Facial Landmarks Detector with dlib

В этом уроке вы узнаете, как создать быстрый детектор лицевых ориентиров с помощью библиотеки dlib.

In this tutorial, you will learn how to create a fast facial landmark detector using the dlib library.

5-точечный детектор лицевых ориентиров состоит из:

The 5-point face landmark detector consists of:

* 2 точек для левого глаза; 2 points for the left eye
* 2 точек для правого глаза; 2 points for the right eye
* 1 точки для носа; 1 points for nose


### Загрузка файлов
### Download the code zip file

In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_29.faster_facial_landmarks.zip
!unzip -qq Lesson_29.faster_facial_landmarks.zip
%cd /content/Lesson_29.faster_facial_landmarks/

--2022-02-08 16:08:36--  http://dataudt.ru/datasets/cv/Lesson_29.faster_facial_landmarks.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13204872 (13M) [application/zip]
Saving to: ‘Lesson_29.faster_facial_landmarks.zip’

Lesson_29.faster_fa 100%[===================>]  12.59M  6.25MB/s    in 2.0s    

2022-02-08 16:08:38 (6.25 MB/s) - ‘Lesson_29.faster_facial_landmarks.zip’ saved [13204872/13204872]

/content/Lesson_29.faster_facial_landmarks


### Импорт библиотек
### Import Packages

In [2]:
# импортируем необходимые библиотеки
# import the necessary packages
from imutils import face_utils
import imutils
import dlib
import cv2

### Реализация детектора лицевых ориентиров с помощью dlib, OpenCV и Python
### Implementing facial landmarks with dlib, OpenCV, and Python

In [3]:

# явно укажем аргументы необходимые для работы
# shape_predictor - путь к предиктору лицевых ориентиров
# input - входное видео 
# output - выходное видео

# explicitly specify the arguments needed to work
# shape_predictor - path to facial landmark predictor
# input - input video 
# output - output video  
args = {
	"shape_predictor": "shape_predictor_5_face_landmarks.dat",
	"input": "Facemask_detection.mp4",
	"output": "output.avi"
}

In [4]:
# инициализируем детектор лиц dlib (на основе HOG), а затем создаем
# детектор лица
# initialize dlib's face detector (HOG-based) and then create the
# facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

# берем ссылку на видео и инициализируем путь к выходному файлу
# grab a reference to the video file and initialize pointer to output
# video file
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["input"])
writer = None

[INFO] loading facial landmark predictor...
[INFO] opening video file...


In [5]:
# перебираем кадры из видео
# loop over the frames from the video stream
while True:
    # возьмем следующий кадр из видео 
    # grab the next frame
	frame = vs.read()[1]

    # если кадр не был захвачен, значит мы дошли до конца видео
	# if we did not grab a frame then we have reached the end of the
	# video
	if frame is None:
		break
    
    # изменим размер кадра на максимальную ширину 400 пикселей
    # и сконвертируем его в оттенки серого
	# resize the frame to have a maximum width of 400 pixels, and
    # convert it to grayscale
	frame = imutils.resize(frame, width=400)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # обнаружаем лиц в рамке с оттенками серого
	# detect faces in the grayscale frame
	rects = detector(gray, 0)

    # проверяем, было ли обнаружено лицо, и если да, то выводим итоговое
	# количество лиц на кадре
	# check to see if a face was detected, and if so, draw the total
	# number of faces on the frame
	if len(rects) > 0:
		text = "{} face(s) found".format(len(rects))
		cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, (0, 0, 255), 2)
  
    # проходим циклом через все обнаружения
    # loop over the face detections
	for rect in rects:
        # вычисляем ограничивающую рамку для лица и рисуем ее на кадре
		# compute the bounding box of the face and draw it on the
		# frame
		(bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
		cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH),
			(0, 255, 0), 1)

        # определяем ориентиры для области лица, затем
		# преобразовываем координаты лицевого ориентира (x, y) в массив NumPy
		# determine the facial landmarks for the face region, then
		# convert the facial landmark (x, y)-coordinates to a NumPy
		# array
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)
 
        # перебираем (x, y) -координаты для лицевых ориентиров
		# и рисуем каждый из них
		# loop over the (x, y)-coordinates for the facial landmarks
		# and draw each of them
		for (i, (x, y)) in enumerate(shape):
			cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
			cv2.putText(frame, str(i + 1), (x - 10, y - 10),
				cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
   
    # если нет видео * И * мы должны записать
    # выходное видео на диск, то инициализируем камеру
    # if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)
  
    # если видео есть, записываем кадр с распознанным лицом на диск
	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)

# очищаем все данные
# do a bit of cleanup
vs.release()

# проверьте, нужно ли очистить запись видео
# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Выполнение ячейки может занять некоторое время. Наше выходное видео создается в формате .avi. Нам нужно преобразовать его в формат .mp4.

The execution of the cell may take some time. Our output video is created in .avi format. We need to convert it to .mp4 format.

In [6]:
!ffmpeg -i output.avi output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [7]:
#@title Воспроизводим выходное видео

#@title Display video inline
from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)